In [14]:
!pip3 install sentencepiece

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 9.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
from tqdm import tqdm
import copy
from transformers import pipeline
import pandas as pd
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import sentencepiece as spm
from tokenizers import Tokenizer
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F

P_df = pd.read_csv("C:\\Emotion Classification\\treeModelsData\\positive_emotions.csv")
N_df = pd.read_csv("C:\\Emotion Classification\\treeModelsData\\negative_emotions.csv")

TOKENIZER = "C:\Emotion Classification\\traning\\vocab\\tokenizer0.2.json"

tokenizer = Tokenizer.from_file(TOKENIZER)

batchsize = 100

<>:16: SyntaxWarning: invalid escape sequence '\E'
<>:16: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_21808\684743216.py:16: SyntaxWarning: invalid escape sequence '\E'
  TOKENIZER = "C:\Emotion Classification\\traning\\vocab\\tokenizer0.2.json"


In [250]:
P_df.shape[0] + N_df.shape[0], P_df.shape[1]

(43332, 14)

In [251]:
P_df.dtypes

text          object
admiration     int64
amusement      int64
approval       int64
caring         int64
curiosity      int64
desire         int64
excitement     int64
gratitude      int64
joy            int64
love           int64
optimism       int64
pride          int64
relief         int64
dtype: object

In [252]:
P_df['text'] = P_df['text'].str.lower()
N_df['text'] = N_df['text'].str.lower()

In [253]:
P_df.sample(1)

,text,admiration,amusement,approval,caring,curiosity,desire,excitement,gratitude,joy,love,optimism,pride,relief
16608,ughh same. love the finish of it just wish it ...,0,0,0,0,0,0,0,0,0,1,0,0,0


In [254]:
split_index = int(P_df.shape[0] * 0.8)
split_index
scaler = MinMaxScaler()

In [272]:
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 128
vocab_size, embedding_dim

(30000, 128)

In [286]:
def pipeline_text(data):
    data = data.copy()
    
    data["text"] = data["text"].apply(lambda x: (tokenizer.encode(x)).ids)
    y = data.drop(columns=['text'])
    
    list_of_lists = data['text'].tolist()
    tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in list_of_lists]
    print(tensor_list[0])
    X = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    print(X[0])
    y = torch.tensor(y.values, dtype=torch.float32)
    
    X_train = X[:split_index]
    X_test = X[split_index:]
    
    y_train = y[:split_index]
    y_test = y[split_index:]
    
    
    return X_train, y_train, X_test, y_test
    
X_train, y_train, X_test, y_test = pipeline_text(P_df)

tensor([ 720,   51,  681, 1141,   18])
tensor([ 720,   51,  681, 1141,   18,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])


In [287]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, type(X_train), type(y_train), type(X_test), type(y_test)

(torch.Size([21153, 62]),
 torch.Size([21153, 13]),
 torch.Size([5289, 62]),
 torch.Size([5289, 13]),
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor)

In [288]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False, num_workers=2)

In [289]:
sequence_len = X_train.shape[1]
input_len = X_train.shape[1]
hidden_size = 256
num_layers = 2
num_classes = y_train.shape[1]
num_epochs = 5
learning_rate = 0.01

In [325]:
# class LSTM(nn.Module):
#     def __init__(self, input_len, hidden_size, num_layers, num_classes):
#         super(LSTM, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
#         self.output_layer = nn.Linear(hidden_size, num_classes) 
    
#     def forward(self, X):
#         hidden_size = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
#         cell_state = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
#         out, _ = self.lstm(X, (hidden_size, cell_state))
#         out = self.output_layer(out[:, -1, :])
#         return out
    
    
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classesб, dropout=0.3):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_size, 
            num_layers=num_layers,  # Збільшуємо кількість LSTM шарів
            batch_first=True, 
            dropout=dropout if num_layers > 1 else 0,  # Dropout між LSTM шарами
            bidirectional=False  # Можна змінити на True при потребі
        )
        self.dropout1 = nn.Dropout(dropout)
        
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn1 = nn.BatchNorm1d(hidden_size // 2)  # Batch normalization
        self.dropout2 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.bn2 = nn.BatchNorm1d(hidden_size // 4)
        self.dropout3 = nn.Dropout(dropout * 0.5)  # Менший dropout в кінці
        
        # Вихідний шар
        self.output_layer = nn.Linear(hidden_size // 4, num_classes)
    
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        
        # LSTM
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        # Використовуємо останній hidden state
        out = lstm_out[:, -1, :]  # Беремо останню позицію
        
        # Dropout після LSTM
        out = self.dropout1(out)
        
        # Перший FC шар + BatchNorm + ReLU + Dropout
        out = self.fc1(out)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.dropout2(out)
        
        # Другий FC шар + BatchNorm + ReLU + Dropout
        out = self.fc2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = self.dropout3(out)
        
        # Вихідний шар
        out = self.output_layer(out)
        
        return out

In [326]:
# model = LSTMModel(input_len, hidden_size, num_layers, num_classes)
model = LSTMModel(vocab_size, hidden_size, hidden_size, num_layers, num_classes)

In [327]:
loss_func = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [328]:
def train(num_epochs, model, train_dataloader, loss_func):
    total_step = len(train_dataloader)
    
    for epoch in range(num_epochs):
        for batch, (text_r, lables) in enumerate(train_dataloader):

            outputs = model(text_r)
            loss = loss_func(outputs, lables)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch + 1}/{total_step}], Loss: {loss.item():.4f}, Accuracy: {torch.sum(torch.argmax(outputs, dim=1) == torch.argmax(lables, dim=1)).item() / len(lables):.4f}')

In [329]:
train(num_epochs=num_epochs, model=model, train_dataloader=train_dataloader, loss_func=loss_func)

NameError: name 'F' is not defined

Epoch [1/5], Step [100/212], Loss: 3.0014, Accuracy: 0.1600
Epoch [1/5], Step [200/212], Loss: 3.0891, Accuracy: 0.1600
Epoch [2/5], Step [100/212], Loss: 3.0311, Accuracy: 0.1800
Epoch [2/5], Step [200/212], Loss: 3.1607, Accuracy: 0.2200
Epoch [3/5], Step [100/212], Loss: 2.8715, Accuracy: 0.2000
Epoch [3/5], Step [200/212], Loss: 2.9521, Accuracy: 0.1900
Epoch [4/5], Step [100/212], Loss: 3.0117, Accuracy: 0.1900
Epoch [4/5], Step [200/212], Loss: 2.7980, Accuracy: 0.2200
Epoch [5/5], Step [100/212], Loss: 2.7254, Accuracy: 0.2200
Epoch [5/5], Step [200/212], Loss: 2.8732, Accuracy: 0.1900